In [ ]:
def initial_global_model(output_folder_name, num_days, seed_number, combined_data_training, combined_data_validation, combined_data_testing, function_name, mins_before_predict_list, window_slide_list, epochs):    
    random.seed(seed_number) 
    np.random.seed(seed_number)
    tf.random.set_seed(seed_number)
    output_folder_models =str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(function_name)+'/models'
    os.makedirs(output_folder_models, exist_ok=True) 


    ########output############
    results_df = pd.DataFrame(columns=['RMSE'])   
    # Extract the input features and target variable
    X_training = combined_data_training[['insulin', 'CHO', 'CGM']]
    y_training = combined_data_training['CGM']
    
    X_validation = combined_data_validation[['insulin', 'CHO', 'CGM']]
    y_validation = combined_data_validation['CGM']
    
    X_testing = combined_data_testing[['insulin', 'CHO', 'CGM']]
    y_testing = combined_data_testing['CGM']
    # Convert the DataFrame to numpy arrays
    X_training= X_training.to_numpy()
    y_training = y_training.to_numpy()
    
    X_validation= X_validation.to_numpy()
    y_validation = y_validation.to_numpy()
    
    X_testing= X_testing.to_numpy()
    y_testing = y_testing.to_numpy()
    # Define the number of timesteps
    for mins_before_predict in mins_before_predict_list:
        timesteps = int(mins_before_predict/3)
        best_model = None

        results = []
        for window_slide in window_slide_list:

            # Create input sequences and corresponding target values
            X_sequences_training = []
            y_targets_training = []

            X_sequences_validation = []
            y_targets_validation = []

            X_sequences_testing = []
            y_targets_testing = []


            for i in range(0, len(X_training) - timesteps-window_slide):
                X_sequences_training.append(X_training[i:i+window_slide])
                y_targets_training.append(y_training[i+timesteps+window_slide-1])

            # Convert sequences and targets to numpy arrays
            X_sequences_training = np.array(X_sequences_training)
            y_targets_training = np.array(y_targets_training)

            for i in range(0, len(X_validation) - timesteps-window_slide):
                X_sequences_validation.append(X_validation[i:i+window_slide])
                y_targets_validation.append(y_validation[i+timesteps+window_slide-1])

            # Convert sequences and targets to numpy arrays
            X_sequences_validation = np.array(X_sequences_validation)
            y_targets_validation = np.array(y_targets_validation)


            for i in range(0, len(X_testing) - timesteps-window_slide):
                X_sequences_testing.append(X_testing[i:i+window_slide])
                y_targets_testing.append(y_testing[i+timesteps+window_slide-1])

            # Convert sequences and targets to numpy arrays
            X_sequences_testing = np.array(X_sequences_testing)
            y_targets_testing = np.array(y_targets_testing)


            results = []
            y_test, y_pred, RMSE = deep_learning_analysis(output_folder_name, seed_number,  function_name, mins_before_predict, window_slide, epochs, batch_size, learning_rate,monitor, neuron_nums, X_sequences_training, y_targets_training, X_sequences_validation, y_targets_validation, X_sequences_testing, y_targets_testing)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            
            new_data = {'RMSE': rmse}
            new_df = pd.DataFrame(new_data, index=[0])
            # Concatenate the existing results_df DataFrame with the new DataFrame
            results_df = pd.concat([results_df, new_df], ignore_index=True)
            model = load_model(os.path.join(output_folder_models, 'best_model_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5'))
            file_name_global = "RNN_"+str(mins_before_predict)+'_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+".h5"
            model.save(os.path.join(output_folder_models, file_name_global))
            return rmse
